This notebooks contains code needed to produce visualization used in thesis

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt


import torch
import torch.autograd.profiler as profiler

torch.cuda.benchmark = False
count = lambda x: sum([p.numel() for p in x.parameters()])

In [36]:
import torch.utils.benchmark as bench

In [37]:
bench.Timer?

In [44]:
in_chs, out_chs = 128, 128
inp = torch.randn(64, in_chs, 224, 224).cuda().half()
conv = torch.nn.Conv2d(in_chs, out_chs, 3, 1, 1, bias=False).cuda().half()

# conv = torch.nn.Sequential(
#     torch.nn.Conv2d(in_chs, out_chs, 3, 1, 1, bias=False, groups=in_chs),
#     torch.nn.Conv2d(in_chs, out_chs, 1, 1, 0, bias=False)
# ).cuda().half()


conv = conv.eval().requires_grad_(False)
for _ in range(100):
    conv(inp) # warmup 
count(conv)

147456

In [45]:
with profiler.profile(record_shapes=False, use_cuda=True) as prof:
#     with torch.cuda.amp.autocast():
    with profiler.record_function("single_conv"):
        for _ in range(100):
            out = conv(inp)
            
prof.export_chrome_trace("trace.json")

In [46]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                single_conv         0.29%       2.187ms        99.99%     754.653ms     754.653ms     597.126us         0.04%        1.539s        1.539s             1  
               aten::conv2d         0.09%     686.067us        99.70%     752.462ms       7.525ms     557.967us         0.04%        1.539s      15.389ms           100  
          aten::convolution         0.09%     652.311us        99.61%     751.776ms       7.518ms     563.865us         0.04%        1.538s      15.3